# Schreier-Sims algorithm

In [1]:
import numpy as np, time
from functions import compose,inverse, compute_schreiers_vector, generating_set, sims_filter

### Choose: 3x3 Rubik's Cube

In [2]:
turns = {
    "E": np.arange(48),
    "R": np.array([28,3,4,5,6,7,8,1,2,9,10,11,12,13,14,15,16,17,18,47,0,41,22,23,24,
            25,26,35,36,37,30,31,32,33,34,19,20,21,38,39,40,29,42,43,44,45,46,27]),
    "L": np.array([0,1,2,3,4,5,6,7,8,11,12,13,14,15,16,9,10,33,18,19,20,21,22,39,40,
            45,26,27,28,29,30,43,44,25,34,35,36,37,38,31,32,41,42,23,24,17,46,47]),
    "U": np.array([0,33,34,35,4,5,6,7,8,41,42,43,12,13,14,15,16,19,20,21,22,23,24,17,18,
            25,26,27,28,29,30,31,32,9,10,11,36,37,38,39,40,1,2,3,44,45,46,47]),
    "D": np.array([0,1,2,3,4,45,46,47,8,9,10,11,12,37,38,39,16,17,18,19,20,21,22,23,24,
            27,28,29,30,31,32,25,26,33,34,35,36,5,6,7,40,41,42,43,44,13,14,15]),
    "F": np.array([0,27,2,3,4,5,6,25,26,9,10,21,22,23,14,15,16,17,18,19,20,7,8,1,24,
            11,12,13,28,29,30,31,32,35,36,37,38,39,40,33,34,41,42,43,44,45,46,47]),
    "B": np.array([42,1,2,17,18,19,6,7,8,31,10,11,12,13,14,29,30,15,16,9,20,21,22,23,24,
            25,26,27,28,3,4,5,32,33,34,35,36,37,38,39,40,43,44,45,46,47,0,41])
}

n = 48

# different generating sets to choose from
rubiks = [turns["R"], turns["L"], turns["U"], turns["D"], turns["F"], turns["B"]]
rubiks_without_B = [turns["R"], turns["L"], turns["U"], turns["D"], turns["F"]]
ru = [compose([turns["R"], turns["U"]], n)]
rup = [compose([turns["R"], inverse(turns["U"], n)], n)]
h = [compose([turns["R"], turns["R"]], n), compose([turns["L"], turns["L"]], n), turns["U"], turns["D"], 
     compose([turns["F"], turns["F"]], n), compose([turns["B"], turns["B"]], n)]
highest_order_element = [compose([turns["R"], turns["U"], turns["U"], inverse(turns["D"], n), turns["B"], inverse(turns["D"], n)], n)]
circle = [compose([turns["R"], turns["U"], turns["L"], turns["D"]], n)]

# choose a generating set
generators = rubiks

# different possiblities of orders in which elements should be stabilized
human_order = np.array([26,28,30,32,25,27,29,31,4,8,12,16,17,19,21,18,20,22,
                   1,2,3,5,6,7,9,10,11,13,14,15,23,24,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,0])
trivial_order = np.arange(48)

# choose a solving order
solving_order = human_order

# some elements of S_n
corner_twist = np.array([0,35,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,1,22,23,24,
            25,26,27,28,29,30,31,32,33,34,21,36,37,38,39,40,41,42,43,44,45,46,47])

### or: 4x4 Rubik's Cube (actually 4x4 Supercube)

In [2]:
turns = {
    "E": np.arange(96),
    "R": np.array([0, 4, 8, 12, 16, 3, 7, 11, 15, 2, 6, 10, 14, 1, 5, 9, 13, 17, 18, 19, 20, 21, 22, 23, 24,
            25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 93, 37, 38, 39, 89, 41, 42, 43, 85, 45, 46, 47, 81,
            49, 50, 51, 68, 53, 54, 55, 72, 57, 58, 59, 76, 61, 62, 63, 80, 65, 66, 67, 36, 69, 70, 71, 40,
            73, 74, 75, 44, 77, 78, 79, 48, 64, 82, 83, 84, 60, 86, 87, 88, 56, 90, 91, 92, 52, 94, 95]),
    "r": np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
            25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 94, 36, 37, 38, 90, 40, 41, 42, 86, 44, 45, 46, 82, 48,
            49, 50, 67, 52, 53, 54, 71, 56, 57, 58, 75, 60, 61, 62, 79, 64, 65, 66, 35, 68, 69, 70, 39, 72,
            73, 74, 43, 76, 77, 78, 47, 80, 81, 63, 83, 84, 85, 59, 87, 88, 89, 55, 91, 92, 93, 51, 95]),
    "L": np.array([33, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 20, 24, 28, 32, 19, 23, 27, 31,
            18, 22, 26, 30, 17, 21, 25, 29, 65, 34, 35, 36, 69, 38, 39, 40, 73, 42, 43, 44, 77, 46, 47, 48,
            0, 50, 51, 52, 92, 54, 55, 56, 88, 58, 59, 60, 84, 62, 63, 64, 49, 66, 67, 68, 53, 70, 71, 72,
            57, 74, 75, 76, 61, 78, 79, 80, 81, 82, 83, 45, 85, 86, 87, 41, 89, 90, 91, 37, 93, 94, 95]),
    "l": np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
            25, 26, 27, 28, 29, 30, 31, 32, 33, 66, 35, 36, 37, 70, 39, 40, 41, 74, 43, 44, 45, 78, 47, 48,
            49, 95, 51, 52, 53, 91, 55, 56, 57, 87, 59, 60, 61, 83, 63, 64, 65, 50, 67, 68, 69, 54, 71, 72,
            73, 58, 75, 76, 77, 62, 79, 80, 81, 82, 46, 84, 85, 86, 42, 88, 89, 90, 38, 92, 93, 94, 34]),
    "U": np.array([0, 65, 66, 67, 68, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 81, 82, 83, 84, 21, 22, 23, 24,
            25, 26, 27, 28, 29, 30, 31, 32, 36, 40, 44, 48, 35, 39, 43, 47, 34, 38, 42, 46, 33, 37, 41, 45,
            49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 17, 18, 19, 20, 69, 70, 71, 72,
            73, 74, 75, 76, 77, 78, 79, 80, 1, 2, 3, 4, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]),
    "u": np.array([0, 1, 2, 3, 4, 69, 70, 71, 72, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 85, 86, 87, 88,
            25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
            49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 21, 22, 23, 24,
            73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 5, 6, 7, 8, 89, 90, 91, 92, 93, 94, 95]),
    "D": np.array([32, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 93, 94, 95, 0, 17, 18, 19, 20, 21, 22, 23, 24,
            25, 26, 27, 28, 77, 78, 79, 80, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
            52, 56, 60, 64, 51, 55, 59, 63, 50, 54, 58, 62, 49, 53, 57, 61, 65, 66, 67, 68, 69, 70, 71, 72,
            73, 74, 75, 76, 13, 14, 15, 16, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 29, 30, 31]),
    "d": np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 89, 90, 91, 92, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
            73, 74, 75, 76, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
            49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
            9, 10, 11, 12, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 25, 26, 27, 28, 93, 94, 95]),
    "F": np.array([0, 52, 2, 3, 4, 51, 6, 7, 8, 50, 10, 11, 12, 49, 14, 15, 16, 17, 18, 19, 48, 21, 22, 23, 47,
            25, 26, 27, 46, 29, 30, 31, 45, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 1, 5, 9, 13,
            20, 24, 28, 32, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 68, 72, 76, 80, 67, 71, 75, 79,
            66, 70, 74, 78, 65, 69, 73, 77, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]),
    "f": np.array([0, 1, 56, 3, 4, 5, 55, 7, 8, 9, 54, 11, 12, 13, 53, 15, 16, 17, 18, 44, 20, 21, 22, 43, 24,
            25, 26, 42, 28, 29, 30, 41, 32, 33, 34, 35, 36, 37, 38, 39, 40, 2, 6, 10, 14, 45, 46, 47, 48,
            49, 50, 51, 52, 19, 23, 27, 31, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
            73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]),
    "B": np.array([93, 1, 2, 3, 33, 5, 6, 7, 34, 9, 10, 11, 35, 13, 14, 15, 36, 61, 18, 19, 20, 62, 22, 23, 24,
            63, 26, 27, 28, 64, 30, 31, 32, 29, 25, 21, 17, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
            49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 16, 12, 8, 4, 65, 66, 67, 68, 69, 70, 71, 72,
            73, 74, 75, 76, 77, 78, 79, 80, 84, 88, 92, 0, 83, 87, 91, 95, 82, 86, 90, 94, 81, 85, 89]),
    "b": np.array([0, 1, 2, 37, 4, 5, 6, 38, 8, 9, 10, 39, 12, 13, 14, 40, 16, 17, 57, 19, 20, 21, 58, 23, 24,
            25, 59, 27, 28, 29, 60, 31, 32, 33, 34, 35, 36, 30, 26, 22, 18, 41, 42, 43, 44, 45, 46, 47, 48,
            49, 50, 51, 52, 53, 54, 55, 56, 15, 11, 7, 3, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
            73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95])
}

n = 96

# different generating sets to choose from
rubiks = [turns["R"], turns["L"], turns["U"], turns["D"], turns["F"], turns["B"], 
          turns["r"], turns["l"], turns["u"], turns["d"], turns["f"], turns["b"]]

rubiks_without_l_b_d = [turns["R"], turns["L"], turns["U"], turns["D"], turns["F"], turns["B"], 
                       turns["r"], turns["u"], turns["f"]]

rubiks_3x3 = [turns["R"], turns["L"], turns["U"], turns["D"], turns["F"], turns["B"]]
ru = [compose([turns["R"], turns["U"]], n)]

# choose a generating set
generators = rubiks

# different possiblities of orders in which elements should be stabilized
human_order = np.array([6,7,10,11,22,23,26,27,38,39,42,43,54,55,58,59,70,71,74,75,86,87,90,91,
                        50,51,56,60,62,63,53,57,49,52,61,64,5,9,8,12,21,25,24,28,34,35,40,44,46,47,41,37,33,36,48,45,
                        13,14,15,16,93,94,95,0,29,30,31,32,77,78,79,80,69,73,72,76,85,
                        89,88,92,1,2,3,4,81,82,83,84,17,18,19,20,65,66,67,68])
trivial_order = np.arange(96)

# choose a solving order
solving_order = human_order

# some elements of S_n
corner_twist = np.array([0,35,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,1,22,23,24,
                         25,26,27,28,29,30,31,32,33,34,21,36,37,38,39,40,41,42,43,44,45,46,47,48,
                         49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,
                         73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96])

### The algorithm

In [3]:
# list of group orders of the stabilizer groups
orders_list = []

# list of generating sets of the stabilizer groups
generators_list = []
length_generators_list_before_sims_filter = []
length_generators_list = []

# list of Schreiers vectors for the stabilizer groups
schreiers_vectors_list = []

# the algorithm
i = 0

length_generators_list_before_sims_filter.append(len(generators))

temp_generators = sims_filter(generators, n)
length_generators_list.append(len(temp_generators))
generators_list.append(temp_generators)

start_time = time.time()

while(len(temp_generators) > 0):
    k = solving_order[i]

    schreiers_vector = compute_schreiers_vector(temp_generators, n, k)
    schreiers_vectors_list.append(schreiers_vector)

    schreiers_vector_non_zero = [element for element in schreiers_vector if np.any(element != 0)]

    order_orbit = len(schreiers_vector_non_zero)
    orders_list.append(order_orbit)

    new_generators = generating_set(temp_generators, schreiers_vector, schreiers_vector_non_zero, n, k)
    length_generators_list_before_sims_filter.append(len(new_generators))
    # print(f"Length of new generators before Sims filter: {len(new_generators)}")

    temp_generators = sims_filter(new_generators, n)
    length_generators_list.append(len(temp_generators))
    generators_list.append(temp_generators)
    # print(f"Length of new generators after Sims filter: {len(temp_generators)}")

    i += 1

    elapsed_time = time.time()-start_time
    print(f"Time after iteration {i}: {elapsed_time:.2f} seconds")


# order of 3x3 rubiks cube group
orders_list = np.array(orders_list, dtype=object)
print(np.prod(orders_list))

Time after iteration 1: 0.01 seconds
Time after iteration 2: 0.97 seconds
Time after iteration 3: 2.77 seconds
Time after iteration 4: 3.90 seconds
Time after iteration 5: 5.16 seconds
Time after iteration 6: 6.06 seconds
Time after iteration 7: 6.76 seconds
Time after iteration 8: 7.20 seconds
Time after iteration 9: 7.55 seconds
Time after iteration 10: 7.74 seconds
Time after iteration 11: 7.89 seconds
Time after iteration 12: 7.95 seconds
Time after iteration 13: 8.01 seconds
Time after iteration 14: 8.03 seconds
Time after iteration 15: 8.03 seconds
Time after iteration 16: 8.04 seconds
Time after iteration 17: 8.04 seconds
Time after iteration 18: 8.04 seconds
43252003274489856000


### Checking whether an element of $S_n$ is contained in the subgroup

In [4]:
# choose the element you want to perform the check on
permutation = corner_twist

contained = True
i = 0
g = permutation

while i < len(schreiers_vectors_list):
    k = solving_order[i]

    orbit_set = schreiers_vectors_list[i]
    h = g[k]
    u = orbit_set[h]
    if np.all(u == 0):
        contained = False
        break
    else:
        g = compose([g, inverse(u, n)], n)
    
    i += 1

if not np.array_equal(g, np.arange(n)):
    contained = False

print(contained)

False
